In [76]:
import os
import platform
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split

In [2]:
os.getcwd()

'/Users/youngseoklee/Desktop/workplace/dacon_energy'

In [58]:
#파일경로 설정
if 'macOS' in platform.platform():
    path = '/Users/youngseoklee/Desktop/workplace/datas/dacon_energy/'
    print(path)
elif 'Linux' in platform.platform():
    path = '/workplace/datas/dacon_energy/'
    print(path)
else:
    print('어느 os에도 속해있지 않습니다')

/Users/youngseoklee/Desktop/workplace/datas/dacon_energy/


In [ ]:
df = pd.read_csv(path + 'train.csv', index_col=0)
test_df = pd.read_csv(path + 'test.csv', index_col=0)
b_info = pd.read_csv(path + 'building_info.csv')

In [187]:
sub = pd.read_csv(path + 'sample_submission.csv', index_col=0)

In [161]:
b_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   건물번호          100 non-null    int64  
 1   건물유형          100 non-null    object 
 2   연면적(m2)       100 non-null    float64
 3   냉방면적(m2)      100 non-null    float64
 4   태양광용량(kW)     100 non-null    object 
 5   ESS저장용량(kWh)  100 non-null    object 
 6   PCS용량(kW)     100 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 5.6+ KB


In [162]:
#float으로 변경
b_info[['태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)']] = b_info[['태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)']].replace('-', '0')

b_info[['태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)']] = b_info[['태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)']].astype(float)

In [163]:
b_info['건물유형'].value_counts()

건물기타       15
공공          8
대학교         8
백화점및아울렛     8
병원          8
상용          8
아파트         8
연구소         8
지식산업센터      8
할인마트        8
호텔및리조트      8
데이터센터       5
Name: 건물유형, dtype: int64

In [164]:
b_info = pd.get_dummies(b_info, columns=['건물유형'])

In [165]:
b_info

,건물번호,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW),건물유형_건물기타,건물유형_공공,건물유형_대학교,건물유형_데이터센터,건물유형_백화점및아울렛,건물유형_병원,건물유형_상용,건물유형_아파트,건물유형_연구소,건물유형_지식산업센터,건물유형_할인마트,건물유형_호텔및리조트
0,1,110634.00,39570.00,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0
1,2,122233.47,99000.00,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0
2,3,171243.00,113950.00,40.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0
3,4,74312.98,34419.62,60.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0
4,5,205884.00,150000.00,0.0,2557.0,1000.0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,93314.00,60500.00,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1
96,97,55144.67,25880.00,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1
97,98,53578.62,17373.75,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1
98,99,53499.00,40636.00,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1


In [166]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 204000 entries, 1_20220601 00 to 100_20220824 23
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   건물번호        204000 non-null  int64  
 1   일시          204000 non-null  object 
 2   기온(C)       204000 non-null  float64
 3   강수량(mm)     43931 non-null   float64
 4   풍속(m/s)     203981 non-null  float64
 5   습도(%)       203991 non-null  float64
 6   일조(hr)      128818 non-null  float64
 7   일사(MJ/m2)   116087 non-null  float64
 8   전력소비량(kWh)  204000 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 15.6+ MB


In [167]:
df.isnull().sum()

건물번호               0
일시                 0
기온(C)              0
강수량(mm)       160069
풍속(m/s)           19
습도(%)              9
일조(hr)         75182
일사(MJ/m2)      87913
전력소비량(kWh)         0
dtype: int64

In [170]:
# Convert '일시' to datetime format
df['일시'] = pd.to_datetime(df['일시'], format='%Y%m%d %H')

# Extract year, month, day, and hour
df['Year'] = df['일시'].dt.year
df['Month'] = df['일시'].dt.month
df['Day'] = df['일시'].dt.day
df['Hour'] = df['일시'].dt.hour


df.drop(['일시', '일조(hr)','일사(MJ/m2)'], axis=1, inplace=True)


In [171]:
train_df = pd.merge(df, b_info, on='건물번호', how='left')

In [172]:
train_df

,건물번호,기온(C),강수량(mm),풍속(m/s),습도(%),전력소비량(kWh),Year,Month,Day,Hour,...,건물유형_대학교,건물유형_데이터센터,건물유형_백화점및아울렛,건물유형_병원,건물유형_상용,건물유형_아파트,건물유형_연구소,건물유형_지식산업센터,건물유형_할인마트,건물유형_호텔및리조트
0,1,18.6,NaN,0.9,42.0,1085.28,2022,6,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,18.0,NaN,1.1,45.0,1047.36,2022,6,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,17.7,NaN,1.5,45.0,974.88,2022,6,1,2,...,0,0,0,0,0,0,0,0,0,0
3,1,16.7,NaN,1.4,48.0,953.76,2022,6,1,3,...,0,0,0,0,0,0,0,0,0,0
4,1,18.4,NaN,2.8,43.0,986.40,2022,6,1,4,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,23.1,NaN,0.9,86.0,881.04,2022,8,24,19,...,0,0,0,0,0,0,0,0,0,1
203996,100,22.4,NaN,1.3,86.0,798.96,2022,8,24,20,...,0,0,0,0,0,0,0,0,0,1
203997,100,21.3,NaN,1.0,92.0,825.12,2022,8,24,21,...,0,0,0,0,0,0,0,0,0,1
203998,100,21.0,NaN,0.3,94.0,640.08,2022,8,24,22,...,0,0,0,0,0,0,0,0,0,1


In [173]:
# Convert '일시' to datetime format
test_df['일시'] = pd.to_datetime(test_df['일시'], format='%Y%m%d %H')

# Extract year, month, day, and hour
test_df['Year'] = test_df['일시'].dt.year
test_df['Month'] = test_df['일시'].dt.month
test_df['Day'] = test_df['일시'].dt.day
test_df['Hour'] = test_df['일시'].dt.hour

# Drop the original '일시' column and 'num_date_time'
test_df.drop(['일시'], axis=1, inplace=True)


In [174]:
test_df = pd.merge(test_df, b_info, on='건물번호', how='left')

In [175]:
X = train_df.drop('전력소비량(kWh)', axis=1)
y = train_df['전력소비량(kWh)']

X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size=0.2, shuffle=False)

In [176]:
import optuna
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from torchmetrics.regression import MeanAbsolutePercentageError
from sklearn.metrics import mean_absolute_percentage_error

In [177]:
def objective(trial):

    param = {
        'objective': 'regression',
        'metric': 'mse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }


    model = lgb.LGBMRegressor(**param)
    model.fit(X_train, y_train, eval_set=[[X_valid, y_valid]], early_stopping_rounds=100, verbose=False)
    y_pred = model.predict(X_valid)


    mape = mean_absolute_percentage_error(y_valid, y_pred)
    print(mape)
    return mape


# Run optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Print best parameters
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2023-08-01 00:03:24,688] A new study created in memory with name: no-name-3e8eba97-5164-4772-b400-3219ec2198f2
/var/folders/qb/s5vw4nk95r53s007_f3ty08m0000gn/T/ipykernel_15260/2951571227.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
/var/folders/qb/s5vw4nk95r53s007_f3ty08m0000gn/T/ipykernel_15260/2951571227.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
/var/folders/qb/s5vw4nk95r53s007_f3ty08m0000gn/T/ipykernel_15260/2951571227.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feat

[LightGBM] [Warning] feature_fraction is set=0.5813993159382741, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5813993159382741
[LightGBM] [Warning] lambda_l1 is set=0.07512994039259038, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.07512994039259038
[LightGBM] [Warning] bagging_fraction is set=0.946850094294572, subsample=1.0 will be ignored. Current value: bagging_fraction=0.946850094294572
[LightGBM] [Warning] lambda_l2 is set=3.410113002496365e-08, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.410113002496365e-08
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3


[I 2023-08-01 00:03:28,470] Trial 0 finished with value: 310302780877287.1 and parameters: {'lambda_l1': 0.07512994039259038, 'lambda_l2': 3.410113002496365e-08, 'num_leaves': 243, 'feature_fraction': 0.5813993159382741, 'bagging_fraction': 0.946850094294572, 'bagging_freq': 3, 'min_child_samples': 45}. Best is trial 0 with value: 310302780877287.1.


310302780877287.1
[LightGBM] [Warning] feature_fraction is set=0.557953233307301, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.557953233307301
[LightGBM] [Warning] lambda_l1 is set=7.804230457362021e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.804230457362021e-05
[LightGBM] [Warning] bagging_fraction is set=0.47610579919622464, subsample=1.0 will be ignored. Current value: bagging_fraction=0.47610579919622464
[LightGBM] [Warning] lambda_l2 is set=1.275563253022607e-08, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.275563253022607e-08
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6


/var/folders/qb/s5vw4nk95r53s007_f3ty08m0000gn/T/ipykernel_15260/2951571227.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
/var/folders/qb/s5vw4nk95r53s007_f3ty08m0000gn/T/ipykernel_15260/2951571227.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
/var/folders/qb/s5vw4nk95r53s007_f3ty08m0000gn/T/ipykernel_15260/2951571227.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.

...

322568746150994.1
[LightGBM] [Warning] feature_fraction is set=0.4591075668280973, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4591075668280973
[LightGBM] [Warning] lambda_l1 is set=5.791011595496539e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5.791011595496539e-08
[LightGBM] [Warning] bagging_fraction is set=0.5405547412528598, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5405547412528598
[LightGBM] [Warning] lambda_l2 is set=0.0010324414096713312, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0010324414096713312
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4


/var/folders/qb/s5vw4nk95r53s007_f3ty08m0000gn/T/ipykernel_15260/2951571227.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
/var/folders/qb/s5vw4nk95r53s007_f3ty08m0000gn/T/ipykernel_15260/2951571227.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
/var/folders/qb/s5vw4nk95r53s007_f3ty08m0000gn/T/ipykernel_15260/2951571227.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.

344495429173984.94
[LightGBM] [Warning] feature_fraction is set=0.4003256690724242, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4003256690724242
[LightGBM] [Warning] lambda_l1 is set=1.5644444614686053e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.5644444614686053e-08
[LightGBM] [Warning] bagging_fraction is set=0.5955035818236802, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5955035818236802
[LightGBM] [Warning] lambda_l2 is set=0.0020349630164876132, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0020349630164876132
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2


/var/folders/qb/s5vw4nk95r53s007_f3ty08m0000gn/T/ipykernel_15260/2951571227.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
/var/folders/qb/s5vw4nk95r53s007_f3ty08m0000gn/T/ipykernel_15260/2951571227.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
/var/folders/qb/s5vw4nk95r53s007_f3ty08m0000gn/T/ipykernel_15260/2951571227.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.

418144424883928.5
Best trial:
  Value:  265579173796455.3
  Params: 
    lambda_l1: 6.556342924809363e-08
    lambda_l2: 0.00012833345059915804
    num_leaves: 8
    feature_fraction: 0.4364304830692286
    bagging_fraction: 0.6998740687369662
    bagging_freq: 1
    min_child_samples: 96


In [178]:
best_params = trial.params

In [179]:
# Create a LGBMRegressor with the best parameters
model = lgb.LGBMRegressor(**best_params)

# Train the model
model.fit(X_train, y_train, eval_set=[[X_valid, y_valid]], early_stopping_rounds=100, verbose=False)
# Make predictions
y_pred = model.predict(X_valid)

# You can calculate the performance of the model using various metrics, for example:
mape = mean_absolute_percentage_error(y_valid, y_pred)
print(f'mape: {mape}')

/Users/youngseoklee/opt/anaconda3/envs/ai/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/youngseoklee/opt/anaconda3/envs/ai/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l1 is set=6.556342924809363e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=6.556342924809363e-08
[LightGBM] [Warning] feature_fraction is set=0.4364304830692286, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4364304830692286
[LightGBM] [Warning] bagging_fraction is set=0.6998740687369662, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6998740687369662
[LightGBM] [Warning] lambda_l2 is set=0.00012833345059915804, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.00012833345059915804
mape: 265579173796455.3


In [180]:
pred = model.predict(test_df)

In [188]:
sub['answer'] = pred

sub.to_csv(path + 'sub.csv')

In [189]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16800 entries, 1_20220825 00 to 100_20220831 23
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   answer  16800 non-null  float64
dtypes: float64(1)
memory usage: 262.5+ KB
